In [1]:
%pylab inline
import pandas as pd
import json

pd.options.display.max_rows = 2000

from geopy.geocoders import Nominatim
geolocator = Nominatim()

from geopy.exc import GeocoderTimedOut

def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)
    
from mtranslate import translate

import re

Populating the interactive namespace from numpy and matplotlib


In [2]:
geolocator.geocode("Debrecen")

GeocoderServiceError: [Errno 11001] getaddrinfo failed

In [21]:
do_geocode("Oradea")

Location(Oradea, Bihor, 410041, România, (47.0569272, 21.9326054, 0.0))

In [22]:
def replace_place(string):
    to_replace=['balan', 'barot', 'csikszereda', 'gyergyo', 'keresztur', 'kezdi',
       'kolozsvar', 'kovaszna', 'parajd', 'regen', 'segesvar', 'szentgyorgy',
       'szovata', 'toplita', 'udvarhely', 'vasarhely', 'vlahica']
    replacement=['Bălan', 'Baraolt', 'Miercurea Ciuc', 'Gheorgheni', 'Cristuru Secuiesc', 'Târgu Secuiesc',
       'Cluj-Napoca', 'Covasna', 'Praid', 'Reghin', 'Sighisoara', 'Sfântu Gheorghe',
       'Sovata', 'Toplița, Harghita', 'Odorheiu Secuiesc', 'Târgu Mureș', 'Vlahița']
    for i in arange(len(to_replace)):
        if string==to_replace[i]: return replacement[i]
    return string

In [23]:
def give_coordinates2(places):
    p_lat=[]
    p_lon=[]
    located_p=[]
    prev_lat=0
    prev_lon=0
    prev_place=""
    prev_place_location=""
    for place in places:
        if(place==prev_place):
            located_p.append(prev_place_location)
            p_lat.append(prev_lat)
            p_lon.append(prev_lon)
        else:
            location = do_geocode(place)
            if(location):
                located_p.append(location[0])
                p_lat.append(location.latitude)
                p_lon.append(location.longitude)
            else: 
                located_p.append(None)
                p_lat.append(0)
                p_lon.append(0)
            prev_place=place
            prev_place_location=located_p[len(located_p)-1]
            prev_lat=p_lat[len(p_lat)-1]
            prev_lon=p_lon[len(p_lon)-1]
    return p_lat,p_lon,located_p

In [24]:
tprev=""
tprev_t=""

In [25]:
def translate_s3(string):
    #print(string,"->")
    global tprev
    global tprev_t
    if isinstance(string, str):
        if(string != tprev):
            tprev=string
            trans=translate(string,'en')
            tprev_t=trans
        string=tprev_t
    #print(string)
    return string

In [26]:
start=time.time()
give_coordinates2(["Oradea","Oradea","Oradea"])
end=time.time()
print(end-start)

0.2835967540740967


In [27]:
start=time.time()
give_coordinates2(["Oradea","București","Constanța"])
end=time.time()
print(end-start)

0.8208060264587402


In [181]:
data=pd.DataFrame.from_dict(json.loads(open("first.json").read()))[48001:]#only first 5000 but these get the location where they live
data2=pd.DataFrame.from_dict(json.loads(open("data.json").read()))#all the data, birth location add

In [182]:
data["birth"]=data["birth"].apply(replace_place)
data2["birth"]=data2["birth"].apply(replace_place)

In [183]:
for i in [0,2]:#birth livesin
    data.sort_values(by=data.columns[i],inplace=True,ascending=False)
    p_lat, p_lon, located_p=give_coordinates2(data[data.columns[i]])
    data[data.columns[i]+"_p_lat"]=p_lat
    data[data.columns[i]+"_p_lon"]=p_lon
    data[data.columns[i]+"_located"]=located_p

In [184]:
def get_countries(places):
    x=[]
    for place in places:
        if(place!=None):
            place_split=re.split("/|, ",place)
            x.append(place_split[len(place_split)-1])
        else: x.append(None)
    return x

Megtalalom az orszagot es leforditom angolra

In [186]:
column="livesin_located"
data["NAME"]=get_countries(data[column])
data.sort_values(by="NAME",inplace=True,ascending=False)
data["NAME"]=[translate_s3(name) for name in data["NAME"]]

In [187]:
all_data=pd.concat([all_data,data])

In [188]:
all_data2 = all_data.drop_duplicates(subset='id')

In [189]:
all_data2.describe()

,id,birth_p_lat,birth_p_lon,livesin_p_lat,livesin_p_lon
count,49208.000000,49208.000000,49208.000000,49208.000000,49208.000000
mean,24608.612116,46.386441,25.273718,41.458219,17.430061
std,14208.546027,0.328592,0.677902,14.498386,14.962882
min,0.000000,45.861728,23.590060,-43.530955,-157.948380
25%,12303.750000,46.075192,24.793776,44.933143,7.540121
50%,24608.500000,46.361404,25.529805,46.304746,23.590060
75%,36912.250000,46.721277,25.802667,46.769337,25.599624
max,49219.000000,46.948127,26.138590,73.966667,174.765551


In [190]:
all_data2.to_json("second.json")